In [43]:
import sys
import os

# Get the absolute path to the project root directory
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the project root directory to the Python path
if root_path not in sys.path:
    sys.path.insert(0, root_path)

from agent.nn import GNN

entities = [bar for foo in list(agent.env.entities.values()) for bar in foo]
relations = (
    agent.env.relations
    + [foo + "_inv" for foo in agent.env.relations]
    + ["current_time", "time_added", "num_recalled", "last_accessed"]
)

gnn = GNN(entities=entities, relations=relations)

/home/tk/.virtualenvs/gnn-agent-new/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.entities to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.entities` for environment variables or `env.get_wrapper_attr('entities')` that will search the reminding wrappers.
  logger.warn(
/home/tk/.virtualenvs/gnn-agent-new/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.relations to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.relations` for environment variables or `env.get_wrapper_attr('relations')` that will search the reminding wrappers.
  logger.warn(


In [2]:
import numpy as np

data = np.array(
    [
        [
            ["agent", "at_location", "room_000", {"current_time": 0}],
            ["agent", "at_location", "room_000", {"current_time": 0}],
            ["room_000", "east", "room_001", {"current_time": 0}],
            ["dep_001", "at_location", "room_000", {"current_time": 0}],
            ["room_000", "west", "wall", {"current_time": 0}],
            ["dep_007", "at_location", "room_000", {"current_time": 0}],
            ["room_000", "north", "wall", {"current_time": 0}],
            ["room_000", "south", "room_004", {"current_time": 0}],
        ]
    ],
    dtype=object,
)
print(data.shape)
gnn(data)

(1, 8, 4)


NameError: name 'gnn' is not defined

In [59]:
import numpy as np

data = np.array(
    [
        [
            [
                "agent",
                "at_location",
                "room_000",
                {"num_recalled": 10, "last_accessed": 0, "time_added": 100},
            ],
            [
                "agent",
                "at_location",
                "room_000",
                {"num_recalled": 10, "last_accessed": 0, "time_added": 100},
            ],
            [
                "agent",
                "at_location",
                "room_000",
                {"num_recalled": 10, "last_accessed": 0, "time_added": 100},
            ],
            ["agent", "at_location", "room_000", {"current_time": 0}],
            ["room_000", "east", "room_001", {"current_time": 0}],
            ["dep_001", "at_location", "room_000", {"current_time": 0}],
            ["room_000", "west", "wall", {"current_time": 0}],
            ["dep_007", "at_location", "room_000", {"current_time": 0}],
            ["room_000", "north", "wall", {"current_time": 0}],
            ["room_000", "south", "room_004", {"current_time": 0}],
        ],
        [
            [
                "agent",
                "at_location",
                "room_000",
                {"num_recalled": 10, "last_accessed": 0, "time_added": 100},
            ],
            [
                "agent",
                "at_location",
                "room_000",
                {"num_recalled": 10, "last_accessed": 0, "time_added": 100},
            ],
            [
                "agent",
                "at_location",
                "room_000",
                {"num_recalled": 10, "last_accessed": 0, "time_added": 100},
            ],
            ["agent", "at_location", "room_000", {"current_time": 0}],
            ["room_000", "east", "room_001", {"current_time": 0}],
            ["room_000", "north", "wall", {"current_time": 0}],
            ["room_000", "south", "room_004", {"current_time": 0}],
        ],
    ],
    dtype=object,
)
print(data.shape)
gnn(data)

(2,)


[tensor([[-0.1889, -0.0842],
         [ 0.0682, -0.0041],
         [-0.1661, -0.1026],
         [ 0.0342,  0.0921],
         [-0.1142, -0.0158],
         [-0.0546, -0.0102],
         [-0.0878, -0.1898]], grad_fn=<SliceBackward0>),
 tensor([[-0.0899, -0.0474],
         [-0.0304, -0.1186],
         [-0.2386, -0.1201],
         [-0.2828, -0.4291]], grad_fn=<SliceBackward0>)]

In [ ]:
import sys
import os

# Get the absolute path to the project root directory
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the project root directory to the Python path
if root_path not in sys.path:
    sys.path.insert(0, root_path)

import yaml
from glob import glob
from tqdm.auto import tqdm
from collections import defaultdict
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", None)

# Collect results grouped by config (excluding seed)
results_by_config = defaultdict(list)

for path in tqdm(glob("../training-results/*/results.yaml")):
    with open(path, "r") as f:
        results = yaml.safe_load(f)

    test_mean = results["test_score"]["mean"]

    with open(path.replace("results.yaml", "train.yaml")) as f:
        hp = yaml.safe_load(f)

    room_size = hp["env_config"]["room_size"]
    qa_policy = hp["qa_policy"]

    explore_policy = hp["explore_policy"]

    mm_long_policy = hp.get("mm_long_policy", None)

    # if explore_policy != "dijkstra":
    #     # Skip non-dijkstra policies for now
    #     continue

    # if mm_long_policy.lower() not in ["fifo", "lru", "lfu"]:
    #     # Skip non-standard MM policies for now
    #     continue

    memory_size = hp.get("max_long_term_memory_size", None)

    config_key = (room_size, qa_policy, explore_policy, mm_long_policy, memory_size)
    results_by_config[config_key].append(test_mean)

# Build a DataFrame from the aggregated results
records = []
for config, scores in sorted(results_by_config.items()):
    room_size, qa_policy, explore_policy, mm_long_policy, memory_size = config
    records.append(
        {
            "room_size": room_size,
            "mean_score": np.mean(scores),
            "std_score": np.std(scores),
            "qa_policy": qa_policy,
            "explore_policy": explore_policy,
            "mm_long_policy": mm_long_policy,
            "memory_size": memory_size,
            "n_seeds": len(scores),
        }
    )

df = pd.DataFrame(records)
pd.set_option("display.precision", 4)

# Display each room size section sorted by mean_score
# for room in df["room_size"].unique():
for room in [
    "xl-different-prob",
    "xxl-different-prob",
]:
    print(f"\n=== Results for room size: {room} ===\n")
    section = df[df["room_size"] == room].drop(columns="room_size")
    section = section.sort_values(by="mean_score", ascending=False).reset_index(
        drop=True
    )
    display(section)
# Export results for each room size to separate JSON files
for room in ["xl-different-prob", "xxl-different-prob"]:
    section = df[df["room_size"] == room].drop(columns="room_size")
    section = section.sort_values(by="mean_score", ascending=False).reset_index(
        drop=True
    )

    # Export to JSON
    filename = f"../data/results_{room}.json"
    section.to_json(filename, orient="records", indent=2)
    print(f"Results for {room} exported to {filename}")

In [1]:
import sys
import os

# Get the absolute path to the project root directory
root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the project root directory to the Python path
if root_path not in sys.path:
    sys.path.insert(0, root_path)

import logging
from agent import LongTermAgent

# Disable logging
logging.getLogger().setLevel(logging.CRITICAL)


agent = LongTermAgent(
    env_config={
        "question_prob": 1.0,
        "seed": 0,
        "terminates_at": 99,
        "randomize_observations": "all",
        "room_size": "xl-different-prob",
        "rewards": {"correct": 1, "wrong": 0, "partial": 0},
        "make_everything_static": False,
        "num_total_questions": 1000,
        "question_interval": 1,
        "include_walls_in_observations": True,
        "deterministic_objects": False,
    },
    qa_policy="most_frequently_used",
    explore_policy="dijkstra",
    mm_long_policy="lfu",
    max_long_term_memory_size=1000,
    num_samples_for_results=1,
    default_root_dir="./foo/",
    save_results=False,
)
agent.test()

/home/tk/.virtualenvs/gnn-agent-new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tk/.virtualenvs/gnn-agent-new/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/gnn-agent-new/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/gnn-agent-new/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:1

KeyboardInterrupt: 

In [2]:
agent.humemai.get_raw_triple_count(), agent.humemai.get_main_triple_count(), agent.humemai.get_memory_count()

(696, 74, 79)

In [3]:
agent.humemai.get_short_term_memory_count(), agent.humemai.get_long_term_memory_count(), agent.humemai.get_main_triple_count(), agent.humemai.get_raw_triple_count()

(5, 74, 74, 696)

In [4]:
agent.humemai.print_raw_triples()

(N91a8f4fa670849d2b342a604488619b3, https://humem.ai/ontology#last_accessed, 2025-01-06T00:00:00)
(N91a8f4fa670849d2b342a604488619b3, http://www.w3.org/1999/02/22-rdf-syntax-ns#predicate, south)
(Nb1eaee988a9849d2985c441882656a73, https://humem.ai/ontology#memoryID, 114)
(N0f3ceee2dfec4068a63a7a31716183c7, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement)
(N06ad05b24962449792dcbcfd0d472468, https://humem.ai/ontology#last_accessed, 2025-01-05T00:00:00)
(N91a8f4fa670849d2b342a604488619b3, http://www.w3.org/1999/02/22-rdf-syntax-ns#subject, room_008)
(Naaa21aca40364526b542cbdf459f4454, https://humem.ai/ontology#memoryID, 102)
(room_017, south, room_021)
(Neb6b8100db5b4c60ac82e20dca8d5261, http://www.w3.org/1999/02/22-rdf-syntax-ns#object, room_027)
(N687f0bf1073248569b2b399bccde35b6, http://www.w3.org/1999/02/22-rdf-syntax-ns#subject, room_026)
(N94caa02ef5d3499490d9edb797dd1c8f, http://www.w3.org/1999/02/22-rdf-syntax-ns#type, http://w

In [5]:
agent.humemai.print_main_triples()

(room_017, south, room_021)
(dep_002, at_location, room_008)
(room_005, east, room_006)
(room_027, east, wall)
(room_006, west, room_005)
(room_000, east, room_001)
(agent, at_location, room_005)
(room_017, north, room_012)
(room_012, east, wall)
(room_021, north, room_017)
(agent, at_location, room_021)
(room_006, east, room_007)
(room_031, north, room_026)
(sta_001, at_location, room_007)
(agent, at_location, room_012)
(room_027, west, room_026)
(dep_001, at_location, room_000)
(room_017, west, room_016)
(room_008, north, room_002)
(room_021, east, wall)
(room_000, south, room_004)
(room_000, north, wall)
(room_011, west, wall)
(room_027, south, wall)
(room_007, south, wall)
(room_011, east, room_012)
(sta_002, at_location, room_008)
(dep_007, at_location, room_000)
(room_026, east, room_027)
(room_007, east, room_008)
(room_021, south, room_027)
(room_031, east, wall)
(agent, at_location, room_007)
(room_012, north, room_009)
(agent, at_location, room_008)
(room_031, west, wall)
(ro

In [ ]:
agent.humemai.print_memories() 

(room_000, west, wall, {'memoryID': '7', 'num_recalled': '0', 'time_added': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(room_000, east, room_001, {'memoryID': '8', 'num_recalled': '0', 'time_added': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(dep_001, at_location, room_000, {'memoryID': '9', 'num_recalled': '1', 'time_added': '2025-01-01T00:00:00', 'last_accessed': '2025-01-10T00:00:00'})
(room_000, north, wall, {'memoryID': '10', 'num_recalled': '0', 'time_added': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(dep_007, at_location, room_000, {'memoryID': '11', 'num_recalled': '4', 'time_added': '2025-01-01T00:00:00', 'last_accessed': '2025-01-10T00:00:00'})
(agent, at_location, room_000, {'memoryID': '12', 'num_recalled': '0', 'time_added': '2025-01-01T00:00:00', 'last_accessed': '2025-01-01T00:00:00'})
(room_000, south, room_004, {'memoryID': '13', 'num_recalled': '0', 'time_added': '2025-01-01T00:00:00', 'last_accessed': '2

In [7]:
agent.humemai.to_list()

[['room_000',
  'west',
  'wall',
  {'memoryID': 7,
   'num_recalled': 0,
   'time_added': '2025-01-01T00:00:00',
   'last_accessed': '2025-01-01T00:00:00'}],
 ['room_000',
  'east',
  'room_001',
  {'memoryID': 8,
   'num_recalled': 0,
   'time_added': '2025-01-01T00:00:00',
   'last_accessed': '2025-01-01T00:00:00'}],
 ['dep_001',
  'at_location',
  'room_000',
  {'memoryID': 9,
   'num_recalled': 1,
   'time_added': '2025-01-01T00:00:00',
   'last_accessed': '2025-01-10T00:00:00'}],
 ['room_000',
  'north',
  'wall',
  {'memoryID': 10,
   'num_recalled': 0,
   'time_added': '2025-01-01T00:00:00',
   'last_accessed': '2025-01-01T00:00:00'}],
 ['dep_007',
  'at_location',
  'room_000',
  {'memoryID': 11,
   'num_recalled': 4,
   'time_added': '2025-01-01T00:00:00',
   'last_accessed': '2025-01-10T00:00:00'}],
 ['agent',
  'at_location',
  'room_000',
  {'memoryID': 12,
   'num_recalled': 0,
   'time_added': '2025-01-01T00:00:00',
   'last_accessed': '2025-01-01T00:00:00'}],
 ['room_00